In [1]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree. 
#

import time
import argparse
import sys, os

import numpy as np
import random

import torch
from torch.autograd import Variable
import torch.nn as nn

from data import get_nli, get_batch, build_vocab
from mutils import get_optimizer, dotdict
from models import NLINet


GLOVE_PATH = "dataset/GloVe/glove.840B.300d.txt"


parser = argparse.ArgumentParser(description='NLI training')
# paths
parser.add_argument("--nlipath", type=str, default='dataset/SNLI/', help="NLI data path (SNLI or MultiNLI)")
parser.add_argument("--outputdir", type=str, default='savedir2/', help="Output directory")
parser.add_argument("--outputmodelname", type=str, default='model.pickle')


# training
parser.add_argument("--n_epochs", type=int, default=20)
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--dpout_model", type=float, default=0., help="encoder dropout")
parser.add_argument("--dpout_fc", type=float, default=0., help="classifier dropout")
parser.add_argument("--nonlinear_fc", type=float, default=0, help="use nonlinearity in fc")
parser.add_argument("--optimizer", type=str, default="sgd,lr=0.1", help="adam or sgd,lr=0.1")
parser.add_argument("--lrshrink", type=float, default=5, help="shrink factor for sgd")
parser.add_argument("--decay", type=float, default=0.99, help="lr decay")
parser.add_argument("--minlr", type=float, default=1e-5, help="minimum lr")
parser.add_argument("--max_norm", type=float, default=5., help="max norm (grad clipping)")

#model
parser.add_argument("--encoder_type", type=str, default='BLSTMEncoder', help="see list of encoders")
parser.add_argument("--enc_lstm_dim", type=int, default=2048, help="encoder nhid dimension")
parser.add_argument("--n_enc_layers", type=int, default=1, help="encoder num layers")
parser.add_argument("--fc_dim", type=int, default=512, help="nhid of fc layers")
parser.add_argument("--n_classes", type=int, default=3, help="entailment/neutral/contradiction")
parser.add_argument("--pool_type", type=str, default='max', help="max or mean")

# gpu
parser.add_argument("--gpu_id", type=int, default=0, help="GPU ID")
parser.add_argument("--seed", type=int, default=1234, help="seed")


params, _ = parser.parse_known_args(" ".split())

# set gpu device
torch.cuda.set_device(params.gpu_id)

# print parameters passed, and all parameters
# print('\ntogrep : {0}\n'.format(sys.argv[1:]))
print(params)


"""
SEED
"""
np.random.seed(params.seed)
torch.manual_seed(params.seed)
torch.cuda.manual_seed(params.seed)

"""
DATA
"""
train, valid, test = get_nli(params.nlipath)
word_vec = build_vocab(train['s1'] + train['s2'] + valid['s1'] + valid['s2'] + test['s1'] + test['s2'], GLOVE_PATH)

for split in ['s1', 's2']:
    for data_type in ['train', 'valid', 'test']:
        eval(data_type)[split] = np.array([['<s>'] + [word for word in sent.split() if word in word_vec] +\
                                          ['</s>'] for sent in eval(data_type)[split]])        

params.word_emb_dim = 300


"""
MODEL
"""
# model config
config_nli_model = {
    'n_words'        :  len(word_vec)          ,
    'word_emb_dim'   :  params.word_emb_dim   ,
    'enc_lstm_dim'   :  params.enc_lstm_dim   ,
    'n_enc_layers'   :  params.n_enc_layers   ,
    'dpout_model'    :  params.dpout_model    ,
    'dpout_fc'       :  params.dpout_fc       ,
    'fc_dim'         :  params.fc_dim         ,
    'bsize'          :  params.batch_size     ,
    'n_classes'      :  params.n_classes      ,
    'pool_type'      :  params.pool_type      ,
    'nonlinear_fc'   :  params.nonlinear_fc   ,
    'encoder_type'   :  params.encoder_type   ,
    'use_cuda'       :  True                  ,

}

# model
encoder_types = ['BLSTMEncoder', 'BLSTMprojEncoder', 'BGRUlastEncoder', 'InnerAttentionMILAEncoder',\
                 'InnerAttentionYANGEncoder', 'InnerAttentionNAACLEncoder', 'ConvNetEncoder', 'LSTMEncoder']
assert params.encoder_type in encoder_types, "encoder_type must be in " + str(encoder_types)
nli_net = NLINet(config_nli_model)
print(nli_net)

Namespace(batch_size=64, decay=0.99, dpout_fc=0.0, dpout_model=0.0, enc_lstm_dim=2048, encoder_type='BLSTMEncoder', fc_dim=512, gpu_id=0, lrshrink=5, max_norm=5.0, minlr=1e-05, n_classes=3, n_enc_layers=1, n_epochs=20, nlipath='dataset/SNLI/', nonlinear_fc=0, optimizer='sgd,lr=0.1', outputdir='savedir2/', outputmodelname='model.pickle', pool_type='max', seed=1234)
** TRAIN DATA : Found 549367 pairs of train sentences.
** DEV DATA : Found 9842 pairs of dev sentences.
** TEST DATA : Found 9824 pairs of test sentences.
Found 38957(/43479) words with glove vectors
Vocab size : 38957
NLINet (
  (encoder): BLSTMEncoder (
    (enc_lstm): LSTM(300, 2048, bidirectional=True)
  )
  (classifier): Sequential (
    (0): Linear (16384 -> 512)
    (1): Linear (512 -> 512)
    (2): Linear (512 -> 3)
  )
)


In [6]:
word_vec

AttributeError: 'dict' object has no attribute 'head'

In [2]:

nli_net = NLINet(config_nli_model)
print(nli_net)

NLINet (
  (encoder): BLSTMEncoder (
    (enc_lstm): LSTM(300, 2048, bidirectional=True)
  )
  (classifier): Sequential (
    (0): Linear (16384 -> 512)
    (1): Linear (512 -> 512)
    (2): Linear (512 -> 3)
  )
)


In [3]:
# loss
weight = torch.FloatTensor(params.n_classes).fill_(1)
loss_fn = nn.CrossEntropyLoss(weight=weight)
loss_fn.size_average = False

# optimizer
optim_fn, optim_params = get_optimizer(params.optimizer)
optimizer = optim_fn(nli_net.parameters(), **optim_params)

# cuda by default
nli_net.cuda()
loss_fn.cuda()
#src_embeddings.cuda()


    
"""
TRAIN
"""
#src_embeddings.volatile = True
val_acc_best = -1e10
adam_stop = False
stop_training = False
lr = optim_params['lr'] if 'sgd' in params.optimizer else None
#index_pad =word2id['<p>']

In [7]:
def trainepoch(epoch):
    print('\nTRAINING : Epoch ' + str(epoch))
    nli_net.train()
    all_costs   = []
    logs        = []
    words_count = 0
    
    last_time = time.time()
    correct = 0.
    # shuffle the data
    permutation = np.random.permutation(len(train['s1']))

    s1 = train['s1'][permutation]
    s2 = train['s2'][permutation]
    target = train['label'][permutation]
    

    optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * params.decay if epoch>1\
                                    and 'sgd' in params.optimizer else optimizer.param_groups[0]['lr']
    print('Learning rate : {0}'.format(optimizer.param_groups[0]['lr']))

    for stidx in range(0, len(s1), params.batch_size):
        # prepare batch
        s1_batch, s1_len = get_batch(s1[stidx:stidx + params.batch_size], word_vec)
        s2_batch, s2_len = get_batch(s2[stidx:stidx + params.batch_size], word_vec)
        s1_batch, s2_batch = Variable(s1_batch.cuda()), Variable(s2_batch.cuda())
        tgt_batch = Variable(torch.LongTensor(target[stidx:stidx + params.batch_size])).cuda()
        k = s1_batch.size(1)  # actual batch size
#         print(s1_batch, s1_len) 
        # model forward
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))

        pred = output.data.max(1)[1]
        correct += pred.long().eq(tgt_batch.data.long()).cpu().sum()
        assert len(pred) == len(s1[stidx:stidx + params.batch_size])
        
        # loss
        loss = loss_fn(output, tgt_batch)
        all_costs.append(loss.data[0])
        words_count += (s1_batch.nelement() + s2_batch.nelement()) / params.word_emb_dim

        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient clipping (off by default)
        shrink_factor = 1
        total_norm = 0
        
        for p in nli_net.parameters():
            if p.requires_grad:
                p.grad.data.div_(k)  # divide by the actual batch size
                total_norm += p.grad.data.norm() ** 2
        total_norm = np.sqrt(total_norm)
        
        if total_norm > params.max_norm:
            shrink_factor = params.max_norm / total_norm
        current_lr = optimizer.param_groups[0]['lr'] # current lr (no external "lr", for adam)
        optimizer.param_groups[0]['lr'] = current_lr * shrink_factor # just for update
        
        # optimizer step
        optimizer.step()
        optimizer.param_groups[0]['lr'] = current_lr

        if len(all_costs) == 100:
            logs.append( '{0} ; loss {1} ; sentence/s {2} ; words/s {3} ; accuracy train : {4}'.format(
                    stidx, round(np.mean(all_costs),2), int(len(all_costs) * params.batch_size / (time.time() - last_time)),
                    int(words_count * 1.0 / (time.time() - last_time)), round(100.*correct/(stidx+k), 2) ))
            print(logs[-1])
            last_time = time.time()
            words_count = 0
            all_costs = []
    train_acc = round(100 * correct/len(s1), 2)
    print('results : epoch {0} ; mean accuracy train : {1}'.format(epoch, train_acc))
    return train_acc

In [8]:
s1 = test['s1'] 

In [9]:
s1

array([ list(['<s>', 'This', 'church', 'choir', 'sings', 'to', 'the', 'masses', 'as', 'they', 'sing', 'joyous', 'songs', 'from', 'the', 'book', 'at', 'a', 'church', '.', '</s>']),
       list(['<s>', 'This', 'church', 'choir', 'sings', 'to', 'the', 'masses', 'as', 'they', 'sing', 'joyous', 'songs', 'from', 'the', 'book', 'at', 'a', 'church', '.', '</s>']),
       list(['<s>', 'This', 'church', 'choir', 'sings', 'to', 'the', 'masses', 'as', 'they', 'sing', 'joyous', 'songs', 'from', 'the', 'book', 'at', 'a', 'church', '.', '</s>']),
       ...,
       list(['<s>', 'A', 'man', 'in', 'a', 'black', 'leather', 'jacket', 'and', 'a', 'book', 'in', 'his', 'hand', 'speaks', 'in', 'a', 'classroom', '.', '</s>']),
       list(['<s>', 'A', 'man', 'in', 'a', 'black', 'leather', 'jacket', 'and', 'a', 'book', 'in', 'his', 'hand', 'speaks', 'in', 'a', 'classroom', '.', '</s>']),
       list(['<s>', 'A', 'man', 'in', 'a', 'black', 'leather', 'jacket', 'and', 'a', 'book', 'in', 'his', 'hand', 'speaks', 

In [10]:
def evaluate(epoch, eval_type='valid', final_eval=False):
    nli_net.eval()
    correct = 0.
    global val_acc_best, lr, stop_training, adam_stop
#     print("val acc best, lr, stop_training, adam_stop: ", val_acc_best, lr, stop_training, adam_stop)
    if eval_type == 'valid':
        print('\nVALIDATION : Epoch {0}'.format(epoch))
    
    s1    = valid['s1']    if eval_type == 'valid' else test['s1']
    s2    = valid['s2']    if eval_type == 'valid' else test['s2']
    target = valid['label'] if eval_type == 'valid' else test['label']

    for i in range(0, len(s1), params.batch_size):
        # prepare batch
        s1_batch, s1_len = get_batch(s1[i:i + params.batch_size], word_vec)
        s2_batch, s2_len = get_batch(s2[i:i + params.batch_size], word_vec)
        s1_batch, s2_batch = Variable(s1_batch.cuda()), Variable(s2_batch.cuda())
        tgt_batch = Variable(torch.LongTensor(target[i:i + params.batch_size])).cuda()
        k = s1_batch.size(1)  # actual batch size
#         print("yo, I am here")
        first_arg = (s1_batch, s1_len)
        second_arg = (s2_batch, s2_len)
        # model forward
        output = nli_net(first_arg, second_arg )
        
        pred = output.data.max(1)[1]
        correct += pred.long().eq(tgt_batch.data.long()).cpu().sum()
        
        
    # save model
    eval_acc  = round(100 * correct / len(s1),2)
    if final_eval:
        print('finalgrep : accuracy {0} : {1}'.format(eval_type, eval_acc))
    else:
        print('togrep : results : epoch {0} ; mean accuracy {1} : {2}'.format(epoch, eval_type, eval_acc))
    
    if eval_type == 'valid' and epoch <= params.n_epochs:
        if eval_acc > val_acc_best:
            print('saving model at epoch {0}'.format(epoch))
            if not os.path.exists(params.outputdir):
                os.makedirs(params.outputdir)
            torch.save(nli_net, os.path.join(params.outputdir, params.outputmodelname))
            val_acc_best = eval_acc
        else:
            if 'sgd' in params.optimizer:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / params.lrshrink
                print('Shrinking lr by : {0}. New lr = {1}'.format(params.lrshrink, optimizer.param_groups[0]['lr']))
                if optimizer.param_groups[0]['lr'] < params.minlr:
                    stop_training = True
            if 'adam' in params.optimizer:
                # early stopping (at 2nd decrease in accuracy)
                stop_training = adam_stop
                adam_stop = True
    return eval_acc

In [11]:
train

{'label': array([1, 2, 0, ..., 1, 2, 0]),
 's1': array([ list(['<s>', 'A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.', '</s>']),
        list(['<s>', 'A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.', '</s>']),
        list(['<s>', 'A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.', '</s>']),
        ...,
        list(['<s>', 'A', 'man', 'is', 'surfing', 'in', 'a', 'bodysuit', 'in', 'beautiful', 'blue', 'water', '.', '</s>']),
        list(['<s>', 'A', 'man', 'is', 'surfing', 'in', 'a', 'bodysuit', 'in', 'beautiful', 'blue', 'water', '.', '</s>']),
        list(['<s>', 'A', 'man', 'is', 'surfing', 'in', 'a', 'bodysuit', 'in', 'beautiful', 'blue', 'water', '.', '</s>'])], dtype=object),
 's2': array([ list(['<s>', 'A', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.', '</s>']),
        list(['<s>', 'A', 'person', 'is', 'at', 'a', 'diner', 

# DONT RUN THE CELL BELOW

In [ ]:

epoch = 1
while not stop_training and epoch <= params.n_epochs:
    train_acc = trainepoch(epoch)
    eval_acc = evaluate(epoch, 'valid')
    epoch += 1


TRAINING : Epoch 1
Learning rate : 0.1
6336 ; loss 70.29 ; sentence/s 115 ; words/s 6686 ; accuracy train : 34.27
332736 ; loss 36.46 ; sentence/s 115 ; words/s 6598 ; accuracy train : 68.7
339136 ; loss 37.81 ; sentence/s 112 ; words/s 6834 ; accuracy train : 68.84
345536 ; loss 36.21 ; sentence/s 115 ; words/s 6617 ; accuracy train : 68.99
351936 ; loss 36.33 ; sentence/s 115 ; words/s 6642 ; accuracy train : 69.14
358336 ; loss 36.69 ; sentence/s 115 ; words/s 6632 ; accuracy train : 69.29
364736 ; loss 36.53 ; sentence/s 115 ; words/s 6622 ; accuracy train : 69.42
371136 ; loss 36.49 ; sentence/s 113 ; words/s 6720 ; accuracy train : 69.55
377536 ; loss 36.33 ; sentence/s 115 ; words/s 6551 ; accuracy train : 69.68
383936 ; loss 36.07 ; sentence/s 115 ; words/s 6626 ; accuracy train : 69.8
390336 ; loss 36.08 ; sentence/s 117 ; words/s 6522 ; accuracy train : 69.92
396736 ; loss 34.88 ; sentence/s 114 ; words/s 6653 ; accuracy train : 70.05
403136 ; loss 36.02 ; sentence/s 115 ; w

In [192]:
os.path.join(params.outputdir, params.outputmodelname)

'savedir2/model.pickle'

In [193]:
'./savedir2/model.pickle'

'./savedir2/model.pickle'

In [194]:
# Run best model on test set.
del nli_net
nli_net = torch.load(os.path.join(params.outputdir, params.outputmodelname))

print('\nTEST : Epoch {0}'.format(epoch))
# evaluate(1e6, 'valid', True)
evaluate(0, 'test', True)

# Save encoder instead of full model
# torch.save(nli_net.encoder, os.path.join(params.outputdir, params.outputmodelname))


TEST : Epoch 2
finalgrep : accuracy test : 80.58


80.58

In [159]:
nli_net

BLSTMEncoder (
  (enc_lstm): LSTM(300, 2048, bidirectional=True)
)

In [160]:
# print('\nTRAINING : Epoch ' + str(epoch))
# nli_net.train()
# all_costs   = []
# logs        = []
# words_count = 0

# last_time = time.time()
# correct = 0.
# # shuffle the data
# permutation = np.random.permutation(len(train['s1']))

# s1 = train['s1'][permutation]
# s2 = train['s2'][permutation]
# target = train['label'][permutation]


# optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * params.decay if epoch>1\
#                                 and 'sgd' in params.optimizer else optimizer.param_groups[0]['lr']
# print('Learning rate : {0}'.format(optimizer.param_groups[0]['lr']))

# for stidx in range(0, len(s1), params.batch_size):
#     # prepare batch
#     s1_batch, s1_len = get_batch(s1[stidx:stidx + params.batch_size], word_vec)
#     s2_batch, s2_len = get_batch(s2[stidx:stidx + params.batch_size], word_vec)
#     s1_batch, s2_batch = Variable(s1_batch.cuda()), Variable(s2_batch.cuda())
#     tgt_batch = Variable(torch.LongTensor(target[stidx:stidx + params.batch_size])).cuda()
#     k = s1_batch.size(1)  # actual batch size
#     print("First positional argument: ", s1_batch, "First positional argument: ", s1_len) 
#     # model forward
#     output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
    
#     first_arg = (s1_batch, s1_len)
#     second_arg = (s2_batch, s2_len)
#     # model forward
#     output = nli_net(first_arg, second_arg )

#     pred = output.data.max(1)[1]
#     correct += pred.long().eq(tgt_batch.data.long()).cpu().sum()
#     assert len(pred) == len(s1[stidx:stidx + params.batch_size])

#     # loss
#     loss = loss_fn(output, tgt_batch)
#     all_costs.append(loss.data[0])
#     words_count += (s1_batch.nelement() + s2_batch.nelement()) / params.word_emb_dim

#     # backward
#     optimizer.zero_grad()
#     loss.backward()

#     # gradient clipping (off by default)
#     shrink_factor = 1
#     total_norm = 0

#     for p in nli_net.parameters():
#         if p.requires_grad:
#             p.grad.data.div_(k)  # divide by the actual batch size
#             total_norm += p.grad.data.norm() ** 2
#     total_norm = np.sqrt(total_norm)

#     if total_norm > params.max_norm:
#         shrink_factor = params.max_norm / total_norm
#     current_lr = optimizer.param_groups[0]['lr'] # current lr (no external "lr", for adam)
#     optimizer.param_groups[0]['lr'] = current_lr * shrink_factor # just for update

#     # optimizer step
#     optimizer.step()
#     optimizer.param_groups[0]['lr'] = current_lr

#     if len(all_costs) == 100:
#         logs.append( '{0} ; loss {1} ; sentence/s {2} ; words/s {3} ; accuracy train : {4}'.format(
#                 stidx, round(np.mean(all_costs),2), int(len(all_costs) * params.batch_size / (time.time() - last_time)),
#                 int(words_count * 1.0 / (time.time() - last_time)), round(100.*correct/(stidx+k), 2) ))
#         print(logs[-1])
#         last_time = time.time()
#         words_count = 0
#         all_costs = []
# train_acc = round(100 * correct/len(s1), 2)
# print('results : epoch {0} ; mean accuracy train : {1}'.format(epoch, train_acc))

In [161]:
eval_type = 'valid'

In [162]:
class NLINet(nn.Module):
    def __init__(self, config):
        super(NLINet, self).__init__()

        # classifier
        self.nonlinear_fc = config['nonlinear_fc']
        self.fc_dim = config['fc_dim']
        self.n_classes = 3
        self.enc_lstm_dim = config['enc_lstm_dim']
        self.encoder_type = config['encoder_type']
        self.dpout_fc = config['dpout_fc']

        self.encoder = eval(self.encoder_type)(config)
        self.inputdim = 4*2*self.enc_lstm_dim
        self.inputdim = 4*self.inputdim if self.encoder_type in \
                        ["ConvNetEncoder", "InnerAttentionMILAEncoder"] else self.inputdim
        self.inputdim = self.inputdim/2 if self.encoder_type=="LSTMEncoder" else self.inputdim
        if self.nonlinear_fc:
            self.classifier = nn.Sequential(
                nn.Dropout(p=self.dpout_fc),
                nn.Linear(self.inputdim, self.fc_dim),
                nn.Tanh(),
                nn.Dropout(p=self.dpout_fc),
                nn.Linear(self.fc_dim, self.fc_dim),
                nn.Tanh(),
                nn.Dropout(p=self.dpout_fc),
                nn.Linear(self.fc_dim, self.n_classes),
                )
        else:
            self.classifier = nn.Sequential(
                nn.Linear(self.inputdim, self.fc_dim),
                nn.Linear(self.fc_dim, self.fc_dim),
                nn.Linear(self.fc_dim, self.n_classes)
                )

    def forward(self, s1, s2):
        # s1 : (s1, s1_len)
        u = self.encoder(s1)
        v = self.encoder(s2)

        features = torch.cat((u, v, torch.abs(u-v), u*v), 1)
        output = self.classifier(features)
        return output

    def encode(self, s1):
        emb = self.encoder(s1)
        return emb

In [163]:
nli_net.eval()
correct = 0.
global val_acc_best, lr, stop_training, adam_stop
#     print("val acc best, lr, stop_training, adam_stop: ", val_acc_best, lr, stop_training, adam_stop)
if eval_type == 'valid':
    print('\nVALIDATION : Epoch {0}'.format(epoch))

s1    = valid['s1']    if eval_type == 'valid' else test['s1']
s2    = valid['s2']    if eval_type == 'valid' else test['s2']
target = valid['label'] if eval_type == 'valid' else test['label']

for i in range(0, len(s1), params.batch_size):
    # prepare batch
    s1_batch, s1_len = get_batch(s1[i:i + params.batch_size], word_vec)
    s2_batch, s2_len = get_batch(s2[i:i + params.batch_size], word_vec)
    s1_batch, s2_batch = Variable(s1_batch.cuda()), Variable(s2_batch.cuda())
    tgt_batch = Variable(torch.LongTensor(target[i:i + params.batch_size])).cuda()
    k = s1_batch.size(1)  # actual batch size
#         print("yo, I am here")
    first_arg = (s1_batch, s1_len)
    second_arg = (s2_batch, s2_len)
    # model forward
    output = nli_net(first_arg, second_arg )

    pred = output.data.max(1)[1]
    correct += pred.long().eq(tgt_batch.data.long()).cpu().sum()


# save model
eval_acc  = round(100 * correct / len(s1),2)
if final_eval:
    print('finalgrep : accuracy {0} : {1}'.format(eval_type, eval_acc))
else:
    print('togrep : results : epoch {0} ; mean accuracy {1} : {2}'.format(epoch, eval_type, eval_acc))

if eval_type == 'valid' and epoch <= params.n_epochs:
    if eval_acc > val_acc_best:
        print('saving model at epoch {0}'.format(epoch))
        if not os.path.exists(params.outputdir):
            os.makedirs(params.outputdir)
        torch.save(nli_net, os.path.join(params.outputdir, params.outputmodelname))
        val_acc_best = eval_acc
    else:
        if 'sgd' in params.optimizer:
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / params.lrshrink
            print('Shrinking lr by : {0}. New lr = {1}'.format(params.lrshrink, optimizer.param_groups[0]['lr']))
            if optimizer.param_groups[0]['lr'] < params.minlr:
                stop_training = True
        if 'adam' in params.optimizer:
            # early stopping (at 2nd decrease in accuracy)
            stop_training = adam_stop
            adam_stop = True
print(eval_acc)


VALIDATION : Epoch 1


TypeError: forward() takes 2 positional arguments but 3 were given